In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/MyDrive/Colab Notebooks/VQ-Dynamics")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

autoregressive_patch_transformer_checkpoints  images_jax
autoregressive_prediction.mp4		      __pycache__
Dynamics.ipynb				      vqvae_checkpoints
encode.py				      vqvae.mp4
failed_mdtransformer_checkpoints	      vqvae.py
failed_transformer_checkpoints


In [ ]:
!nvidia-smi

Tue Jul 22 09:56:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   57C    P0             21W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import jax
import jax.numpy as jnp
import flax.linen as nn
from flax.training import train_state, checkpoints
import optax
import numpy as np
import os
from functools import partial
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from vqvae import (
    generate_image as generate_image_numpy,
    VQVAETrainState,
    create_dataset,
    generate_image,
    eval_step,
    train_step as vq_train_step,
    VQVAE,
)

In [ ]:
vq_latent_dim = 32
vq_num_embeddings = 512
vq_commitment_cost = 0.25
npix = 128
train_vq = False

## Vector-Quantised Variational Auto-Encoder Training

In [ ]:
def vq_training(
    batch_size=32,
    num_epochs=10,
    lr=1e-3,
    latent_dim=32,
    num_embeddings=512,
    commitment_cost=0.25,
    npix=128,
    vq_directory='./vqvae_checkpoints',
):
    # --- Define an absolute directory for checkpoints ---
    ckpt_dir = os.path.abspath(vq_directory)
    os.makedirs(ckpt_dir, exist_ok=True)

    # --- Initialization ---
    key = jax.random.PRNGKey(42)
    key, model_key, data_key = jax.random.split(key, 3)

    model = VQVAE(
        latent_dim=latent_dim,
        num_embeddings=num_embeddings,
        commitment_cost=commitment_cost
    )

    # Initialize model parameters
    dummy_input = jnp.ones((1, npix, npix, 1))
    params = model.init(model_key, dummy_input)['params']

    # Initialize optimizer and state
    optimizer = optax.adam(learning_rate=lr)
    state = VQVAETrainState.create(apply_fn=model.apply, params=params, tx=optimizer)

    # --- Restore checkpoint if one exists ---
    # This will load the latest checkpoint and update the state object in place.
    # If no checkpoint exists, it returns the original state object.
    state = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=state)
    print(f"Resuming training from step: {int(state.step)}")

    # --- Training Loop ---
    print("Starting VQ-VAE training...")
    for epoch in range(num_epochs):
        # Create a new data generator for each epoch
        train_loader = create_dataset(npix, batch_size, size=10000)

        # Use tqdm for a progress bar
        pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", total=10000 // batch_size)

        epoch_metrics = { 'loss': [], 'recon_loss': [], 'vq_loss': [], 'perplexity': [] }

        for batch in pbar:
            key, train_key = jax.random.split(key)
            state, metrics = vq_train_step(state, batch, train_key, model)
            for k, v in metrics.items():
                epoch_metrics[k].append(v)

            pbar.set_postfix(
                loss=f"{metrics['loss']:.4f}",
                recon_loss=f"{metrics['recon_loss']:.4f}",
                perp=f"{metrics['perplexity']:.2f}"
            )

        # --- Save checkpoint at the end of the epoch ---
        checkpoints.save_checkpoint(
            ckpt_dir=ckpt_dir,
            target=state,
            step=state.step,
            overwrite=False, # Set to True to save only the latest checkpoint
            keep=3          # Keep the 3 most recent checkpoints
        )

        # Print average metrics for the epoch
        avg_loss = np.mean([float(x) for x in epoch_metrics['loss']])
        avg_recon_loss = np.mean([float(x) for x in epoch_metrics['recon_loss']])
        avg_vq_loss = np.mean([float(x) for x in epoch_metrics['vq_loss']])
        avg_perp = np.mean([float(x) for x in epoch_metrics['perplexity']])

        print(f"Epoch {epoch + 1} Avg: Loss={avg_loss:.4f}, Recon Loss={avg_recon_loss:.4f}, VQ Loss={avg_vq_loss:.4f}, Perplexity={avg_perp:.4f}")


def vq_reconstruction_animation(state, model):
    # --- Animation Generation ---
    print("\n🎥 Generating animation...")
    os.makedirs('images_jax', exist_ok=True)

    num_points = 200
    num_frames = 100
    sigma = 0.01
    x_coords = np.linspace(0, 1, npix, endpoint=False) + 1 / (2 * npix)
    xx, yy = np.meshgrid(x_coords, x_coords)

    points = np.random.uniform(0, 1, (num_points, 2))
    velocities = np.random.uniform(-0.005, 0.005, (num_points, 2))

    orig_imgs = []
    recon_imgs = []

    for i in tqdm(range(num_frames), desc="Generating frames"):
        orig_img = generate_image(points, xx, yy, sigma, npix)
        orig_imgs.append(orig_img)
        plt.imsave(f'images_jax/orig_frame_{i:04d}.png', orig_img, cmap='gray')

        # Prepare input tensor: (1, H, W, 1)
        input_tensor = jnp.array(orig_img)[jnp.newaxis, ..., jnp.newaxis]
        recon = eval_step(state, input_tensor, model)

        # Convert JAX array back to NumPy for plotting
        recon_img = np.array(recon.squeeze())
        recon_imgs.append(recon_img)
        plt.imsave(f'images_jax/recon_frame_{i:04d}.png', recon_img, cmap='gray')

        # Update points with bounce
        points += velocities
        out_x = (points[:, 0] < 0) | (points[:, 0] > 1)
        velocities[out_x, 0] *= -1
        out_y = (points[:, 1] < 0) | (points[:, 1] > 1)
        velocities[out_y, 1] *= -1


    # Create and save side-by-side animation
    fig, (ax_orig, ax_recon, ax_both) = plt.subplots(1, 3, figsize=(15, 5))
    ax_orig.set_title('Original')
    ax_orig.axis('off')
    ax_recon.set_title('Reconstructed')
    ax_recon.axis('off')
    ax_both.set_title('Original (R) Reconstructed (G, B)')
    ax_both.axis('off')
    plt.tight_layout()

    def col_im(i):
        return np.stack([orig_imgs[i], recon_imgs[i], recon_imgs[i]], axis=-1)

    im_orig = ax_orig.imshow(orig_imgs[0], cmap='gray', animated=True)
    im_recon = ax_recon.imshow(recon_imgs[0], cmap='gray', animated=True)
    im_both = ax_both.imshow(col_im(0), cmap='gray', animated=True)

    def update(frame):
        im_orig.set_array(orig_imgs[frame])
        im_recon.set_array(recon_imgs[frame])
        im_both.set_array(col_im(frame))
        return [im_orig, im_recon, im_both]

    ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=50, blit=True)
    # ani.save('vqvae_animation_jax.gif', writer='pillow')
    ani.save('vqvae.mp4', writer='ffmpeg', fps=20)
    plt.close(fig)

In [ ]:
if train_vq:
  vq_training(
      batch_size=32,
      num_epochs=20,
      lr=1e-3,
      latent_dim=vq_latent_dim,
      num_embeddings=vq_num_embeddings,
      commitment_cost=vq_commitment_cost,
      npix=128,
      vq_directory='./vqvae_checkpoints',
  )

## Shared Functions

In [ ]:
def load_vq():
    print("🧊 Loading pre-trained VQ-VAE model...")
    vqvae_model = VQVAE(
        latent_dim=vq_latent_dim, num_embeddings=vq_num_embeddings, commitment_cost=vq_commitment_cost
    )
    key = jax.random.PRNGKey(42)
    key, vq_key = jax.random.split(key)
    dummy_vq_input = jnp.ones((1, npix, npix, 1))
    vq_params = vqvae_model.init(vq_key, dummy_vq_input)['params']
    vqvae_state = train_state.TrainState.create(
        apply_fn=vqvae_model.apply, params=vq_params, tx=optax.adam(1e-3)
    )
    vq_ckpt_dir = os.path.abspath('./vqvae_checkpoints')
    vqvae_state = checkpoints.restore_checkpoint(ckpt_dir=vq_ckpt_dir, target=vqvae_state)
    if vqvae_state is None:
        raise FileNotFoundError(f"No VQ-VAE checkpoint found in {vq_ckpt_dir}")
    print("✅ VQ-VAE model loaded successfully.")
    return vqvae_state, vqvae_model

vqvae_state, vqvae_model = load_vq()

🧊 Loading pre-trained VQ-VAE model...


✅ VQ-VAE model loaded successfully.


In [ ]:
vq_reconstruction_animation(vqvae_state, vqvae_model)

In [ ]:
def get_trajectory(num_frames):
    x_coords_np = np.linspace(0, 1, npix, endpoint=False) + .5 / npix
    xx_np, yy_np = np.meshgrid(x_coords_np, x_coords_np)
    points_np = np.random.uniform(0, 1, (num_points, 2))
    velocities_np = np.random.uniform(-0.005, 0.005, (num_points, 2))
    orig_imgs = []

    for _ in range(num_frames):
        frame = generate_image_numpy(points_np, xx_np, yy_np, sigma=0.01, npix=npix)
        orig_imgs.append(frame)
        points_np += velocities_np
        out_x = (points_np[:, 0] < 0) | (points_np[:, 0] > 1)
        velocities_np[out_x, 0] *= -1
        out_y = (points_np[:, 1] < 0) | (points_np[:, 1] > 1)
        velocities_np[out_y, 1] *= -1

    all_frames = jnp.array(np.stack(orig_imgs, axis=0)[..., np.newaxis])
    return all_frames, orig_imgs


def get_tokeniser_fns(vqvae_state):
    get_tokens_fn = jax.jit(
      lambda imgs: vqvae_state.apply_fn(
          {'params': vqvae_state.params},
          imgs,
          method=VQVAE.encode_to_indices
      ).reshape(imgs.shape[0], -1)
    )
    decode_fn = jax.jit(
      lambda indices: vqvae_state.apply_fn(
          {'params': vqvae_state.params},
          indices,
          method=VQVAE.decode_from_indices
      )
    )
    return get_tokens_fn, decode_fn


def animate_prediction(
    animation_path, orig_imgs, all_frames, all_tokens, predicted_token_grids, get_tokens_fn, decode_fn,
    history_len,
):
    print("🎥 Generating prediction and creating animation...")
    num_frames = len(all_frames)
    num_predictions = num_frames - history_len
    initial_frames = all_frames[:history_len]
    initial_tokens = get_tokens_fn(initial_frames).flatten()

    # Decode all predicted token grids into images in a single batch
    vmapped_decode_fn = jax.vmap(decode_fn)
    predicted_frames = vmapped_decode_fn(predicted_token_grids.reshape(num_predictions, 16, 16))

    check_decoder = vmapped_decode_fn(all_tokens.reshape(num_predictions+history_len, 16, 16))

    # Combine history with predictions
    pred_imgs = orig_imgs[:history_len] + [np.array(frame.squeeze()) for frame in predicted_frames]

    # --- Create and save the final animation ---
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    titles = [
        'Ground Truth',
        'Transformer Prediction',
        'Truth (R), Prediction (G, B)',
        'Encode-Decode',
    ]
    im_data = [
        orig_imgs,
        pred_imgs,
        np.stack([orig_imgs, pred_imgs, pred_imgs], axis=-1),
        check_decoder,
    ]
    ims = []
    for ax, title, x in zip(axes.flatten(), titles, im_data):
        ax.set_title(title)
        ax.axis('off')
        ims.append(ax.imshow(x[0], cmap='gray', animated=True, vmin=0, vmax=1))
    plt.tight_layout()

    def update(frame):
        for im, data in zip(ims, im_data):
            im.set_array(data[frame])
        return ims

    ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=50, blit=True)
    ani.save(animation_path, writer='ffmpeg', fps=20, dpi=100)
    plt.close(fig)
    print(f"✅ Animation saved to '{animation_path}'")
    return predicted_frames, check_decoder


def metrics(predicted_tokens, all_tokens, decoded_imgs_pred, decoded_imgs_gt):
    perc_correct = []
    for pred, gt in zip(predicted_tokens, all_tokens):
        perc_correct.append((pred == gt).mean().item() * 100)
    image_losses = []
    for pred, gt in zip(decoded_imgs_pred, decoded_imgs_gt):
        image_losses.append(np.mean((pred - gt) ** 2))
    out = {"Tokens Correct (%)": np.array(perc_correct), "Frame Losses": np.array(image_losses)}
    for k, vs in out.items():
      for i, v in enumerate(vs):
        print(f"{k} in the {i}th predicted frame: {v}%")
    return out


def get_batch(points, velocities, history_len, vqvae_state):
    def step_fn(carry, _):
        p, v = carry
        p_next = p + v
        v_next = jnp.where((p_next < 0) | (p_next > 1), -v, v)
        return (p_next, v_next), p_next

    _, points_sequence = jax.lax.scan(step_fn, (points, velocities), None, length=history_len + 1)
    points_sequence = jnp.transpose(points_sequence, (1, 0, 2, 3))

    x_coords = jnp.linspace(0, 1, npix, endpoint=False) + .5 / npix
    xx, yy = jnp.meshgrid(x_coords, x_coords)
    sigma = 0.01

    vmapped_gen_images = jax.vmap(jax.vmap(generate_images_from_points_jax, in_axes=(0, None, None, None)), in_axes=(0, None, None, None))
    images_sequence = vmapped_gen_images(points_sequence, xx, yy, sigma)
    images_sequence = images_sequence[..., jnp.newaxis]

    def get_tokens(images):
        return vqvae_state.apply_fn({'params': vqvae_state.params}, images, method=VQVAE.encode_to_indices)

    vmapped_get_tokens = jax.vmap(jax.vmap(get_tokens))
    token_indices = vmapped_get_tokens(images_sequence)
    batch = token_indices.reshape(token_indices.shape[0], -1)
    return batch


def generate_images_from_points_jax(points, xx, yy, sigma):
    """ JAX implementation of image generation, designed to be vectorized. """
    pixel_grid = jnp.stack([xx, yy], axis=-1)
    points = points[:, jnp.newaxis, jnp.newaxis, :]
    pixel_grid = pixel_grid[jnp.newaxis, :, :, :]
    dist_sq = jnp.sum((pixel_grid - points) ** 2, axis=-1)
    img = jnp.sum(jnp.exp(-dist_sq / (2 * sigma**2)), axis=0)
    img_max = jnp.max(img)
    img = jnp.where(img_max > 0, img / img_max, img)
    return img


def create_point_cloud_dataset(batch_size, history_len, num_points, num_samples=5000):
    """ Generator that yields batches of initial points and velocities. """
    num_batches = num_samples // batch_size
    for _ in range(num_batches):
        points_batch = np.random.uniform(0, 1, (batch_size, num_points, 2))
        velocities_batch = np.random.uniform(-0.005, 0.005, (batch_size, num_points, 2))
        yield jnp.array(points_batch), jnp.array(velocities_batch)

In [ ]:
num_points = 200
history_len = 4
tokens_per_frame = 16 * 16
max_seq_len = (history_len + 1) * tokens_per_frame
transformer_embed_dim = 256
transformer_num_heads = 4
transformer_num_layers = 4
transformer_lr = 1e-4
transformer_batch_size = 256
transformer_epochs = 1000
top_k_sampling = 50
sampling_temperature = 1.2
key = jax.random.PRNGKey(42)

## Auto-Regressive Patch Transformer

In [ ]:
class TokenTransformer(nn.Module):
    num_embeddings: int
    num_heads: int
    num_layers: int
    embed_dim: int
    max_len: int

    @nn.compact
    def __call__(self, x, training: bool):
        causal_mask = nn.make_causal_mask(x)
        tok_emb = nn.Embed(num_embeddings=self.num_embeddings, features=self.embed_dim)(x)
        pos_emb = self.param('pos_embedding', nn.initializers.normal(stddev=0.02), (1, self.max_len, self.embed_dim))
        x = tok_emb + pos_emb[:, :x.shape[1], :]
        x = nn.Dropout(rate=0.1)(x, deterministic=not training)

        for _ in range(self.num_layers):
            x_norm = nn.LayerNorm()(x)
            attn_output = nn.SelfAttention(
                num_heads=self.num_heads,
                dtype=jnp.bfloat16 # Use mixed precision for performance on A100
            )(x_norm, mask=causal_mask)
            attn_output = nn.Dropout(rate=0.1)(attn_output, deterministic=not training)
            x = x + attn_output
            x_norm = nn.LayerNorm()(x)
            y = nn.Dense(features=self.embed_dim * 4)(x_norm)
            y = nn.gelu(y)
            ff_output = nn.Dense(features=self.embed_dim)(y)
            ff_output = nn.Dropout(rate=0.1)(ff_output, deterministic=not training)
            x = x + ff_output
        x = nn.LayerNorm()(x)
        logits = nn.Dense(features=self.num_embeddings)(x)
        return logits


@partial(jax.jit, static_argnums=(5, 6, 7, 8))
def train_step(transformer_state, vqvae_state, points, velocities, key, model, history_len, npix, num_points):
    """ Fully JIT-compiled training step, including data generation. """
    dropout_key, data_key = jax.random.split(key)
    dropout_key = jax.random.fold_in(dropout_key, transformer_state.step)

    batch = get_batch(points, velocities, history_len, vqvae_state)

    inputs = batch[:, :-1]
    targets = batch[:, 1:]

    def loss_fn(params):
        logits = model.apply({'params': params}, inputs, training=True, rngs={'dropout': dropout_key})
        loss = optax.softmax_cross_entropy_with_integer_labels(
            logits=logits, labels=targets
        ).mean()
        return loss

    loss, grads = jax.value_and_grad(loss_fn)(transformer_state.params)
    transformer_state = transformer_state.apply_gradients(grads=grads)
    return transformer_state, loss

In [ ]:
# --- Initialize Transformer Model ---
print("🤖 Initializing Transformer model...")
transformer_model = TokenTransformer(
    num_embeddings=vq_num_embeddings, num_heads=transformer_num_heads, num_layers=transformer_num_layers,
    embed_dim=transformer_embed_dim, max_len=max_seq_len
)
key, transformer_key, dropout_key = jax.random.split(key, 3)
dummy_transformer_input = jnp.ones((1, max_seq_len - 1), dtype=jnp.int32)
transformer_params = transformer_model.init(
    {'params': transformer_key, 'dropout': dropout_key}, dummy_transformer_input, training=False
)['params']
transformer_state = train_state.TrainState.create(
    apply_fn=transformer_model.apply, params=transformer_params, tx=optax.adam(learning_rate=transformer_lr)
)

# --- Define an absolute directory for checkpoints ---
ckpt_dir = os.path.abspath('./autoregressive_patch_transformer_checkpoints')
os.makedirs(ckpt_dir, exist_ok=True)

# --- Restore checkpoint if one exists ---
# This will load the latest checkpoint and update the state object in place.
# If no checkpoint exists, it returns the original state object.
transformer_state = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=transformer_state)
print(f"Resuming training from step: {int(transformer_state.step)}")

🤖 Initializing Transformer model...


Resuming training from step: 18069


In [ ]:
# # Check the checkpoint has loaded:
# data_loader = create_point_cloud_dataset(128, history_len, num_points)
# pbar = tqdm(data_loader)
# total_loss = 0
# for i, (points, velocities) in enumerate(pbar):
#     key, train_key = jax.random.split(key)
#     transformer_state, loss = train_step(
#         transformer_state, vqvae_state, points, velocities, train_key, transformer_model,
#         history_len, npix, num_points
#     )
#     total_loss += loss
# print(f"Avg Loss: {total_loss / (i+1):.4f}")

39it [01:14,  1.91s/it]


Avg Loss: 1.8119


In [ ]:
# --- Phase 3: Train the Transformer ---
print("🚀 Starting Transformer training...")
for epoch in range(transformer_epochs):
    data_loader = create_point_cloud_dataset(
        transformer_batch_size, history_len, num_points
    )
    pbar = tqdm(data_loader, desc=f"Epoch {epoch + 1}/{transformer_epochs}")
    total_loss = 0
    for i, (points, velocities) in enumerate(pbar):
        key, train_key = jax.random.split(key)
        transformer_state, loss = train_step(
            transformer_state, vqvae_state, points, velocities, train_key, transformer_model,
            history_len, npix, num_points
        )
        total_loss += loss
        pbar.set_postfix(loss=f"{loss:.4f}")

    if epoch % 50 == 0:
        # --- Save checkpoint at the end of the epoch ---
        checkpoints.save_checkpoint(
            ckpt_dir=ckpt_dir,
            target=transformer_state,
            step=transformer_state.step,
            overwrite=False, # Set to True to save only the latest checkpoint
            keep=3          # Keep the 3 most recent checkpoints
        )
    print(f"Epoch {epoch + 1} Avg Loss: {total_loss / (i+1):.4f}")


🚀 Starting Transformer training...


Epoch 1/1000: 19it [00:46,  2.47s/it, loss=5.0022]


Epoch 1 Avg Loss: 5.4344


Epoch 2/1000: 19it [00:07,  2.45it/s, loss=4.7136]


Epoch 2 Avg Loss: 4.8299


...

Epoch 998 Avg Loss: 1.8032


Epoch 999/1000: 19it [00:07,  2.44it/s, loss=1.8004]


Epoch 999 Avg Loss: 1.8034


Epoch 1000/1000: 19it [00:07,  2.45it/s, loss=1.8035]

Epoch 1000 Avg Loss: 1.8019


In [ ]:
@partial(jax.jit, static_argnums=(2, 3, 4, 6, 7))
def generate_patch_autoregressive_trajectory(
    transformer_state, initial_history_tokens, num_predictions, history_len, tokens_per_frame, key, top_k, temperature):
    """generates a trajectory with a nested scan, updating the frame-level
    history at each step.
    """
    # The context length for the history (e.g., 4 frames)
    history_context_len = history_len * tokens_per_frame

    # --- This inner function generates one full frame ---
    def predict_one_frame(history_tokens, key):
        # The buffer for this frame's prediction starts with the history
        # and has padding for the new frame to be filled in.
        initial_buffer = jnp.concatenate([
            history_tokens,
            jnp.zeros(tokens_per_frame, dtype=jnp.int32)
        ])

        # The inner scan body predicts a single token
        def predict_one_token(buffer, i):
            step_key = jax.random.fold_in(key, i) # Create a unique key for each token

            # The input to the model is always a fixed-size slice of the buffer
            model_input = buffer[:-1]

            logits = transformer_state.apply_fn(
                {'params': transformer_state.params},
                model_input[jnp.newaxis, :],
                training=False, rngs={'dropout': step_key}
            )

            # We predict the token at the current position `i` within the new frame
            logit_index = history_context_len + i - 1
            next_token_logits = logits[:, logit_index, :]

            # --- Temperature and Top-K Sampling ---
            scaled_logits = next_token_logits / temperature
            top_k_logits, _ = jax.lax.top_k(scaled_logits, k=top_k)
            kth_logit = jnp.min(top_k_logits)
            masked_logits = jnp.where(scaled_logits < kth_logit, -jnp.inf, scaled_logits)

            next_token = jax.random.categorical(step_key, masked_logits)

            # Update the buffer with the predicted token. The shape remains constant.
            updated_buffer = buffer.at[logit_index + 1].set(next_token[0])

            # The updated buffer is the carry; we don't need to collect a separate output
            return updated_buffer, None

        # Run the inner scan to fill in the new frame's tokens
        final_buffer, _ = jax.lax.scan(
            predict_one_token,
            initial_buffer,
            jnp.arange(tokens_per_frame)
        )

        # Return only the newly generated tokens
        return final_buffer[-tokens_per_frame:]

    # --- This outer function manages the frame-level history progression ---
    def generate_next_frame_and_update_history(history_buffer, key):
        # Predict the next full frame based on the current history
        predicted_frame = predict_one_frame(history_buffer, key)

        # Create the new history for the *next* iteration by sliding the window
        new_history_buffer = jnp.concatenate([history_buffer[tokens_per_frame:], predicted_frame])

        return new_history_buffer, predicted_frame

    # --- Run the outer scan to generate all predicted frames ---
    keys = jax.random.split(key, num_predictions)
    _, all_predicted_frames = jax.lax.scan(
        generate_next_frame_and_update_history,
        initial_history_tokens,
        keys
    )

    return all_predicted_frames

In [ ]:
all_frames, orig_imgs = get_trajectory(num_frames=100)

get_tokens_fn, decode_fn = get_tokeniser_fns(vqvae_state)

all_tokens = get_tokens_fn(all_frames)

autoregressive_predicted_token_grids = generate_patch_autoregressive_trajectory(
    transformer_state, all_tokens[:history_len].flatten(), len(all_tokens)-history_len,
    history_len, tokens_per_frame, key, top_k_sampling, sampling_temperature
)

predicted_frames, check_decoder = animate_prediction(
    'autoregressive_predicted_trajectory.mp4',
    orig_imgs,
    all_frames,
    all_tokens,
    autoregressive_predicted_token_grids,
    get_tokens_fn,
    decode_fn,
    history_len,
)

🎥 Generating prediction and creating animation...
✅ Animation saved to 'autoregressive_predicted_trajectory.mp4'


In [ ]:
autoregressive_metrics = metrics(
    autoregressive_predicted_token_grids,
    all_tokens[history_len:],
    predicted_frames,
    check_decoder[history_len:],
)
np.savez_compressed("autoregressive_metrics.npz", **autoregressive_metrics)

Tokens Correct (%) in the 0th predicted frame: 39.0625%
Tokens Correct (%) in the 1th predicted frame: 28.515625%
Tokens Correct (%) in the 2th predicted frame: 22.265625%
Tokens Correct (%) in the 3th predicted frame: 18.75%
Tokens Correct (%) in the 4th predicted frame: 14.453125%
Tokens Correct (%) in the 5th predicted frame: 10.546875%
Tokens Correct (%) in the 6th predicted frame: 7.03125%
Tokens Correct (%) in the 7th predicted frame: 5.078125%
Tokens Correct (%) in the 8th predicted frame: 5.46875%
Tokens Correct (%) in the 9th predicted frame: 5.078125%
Tokens Correct (%) in the 10th predicted frame: 3.90625%
Tokens Correct (%) in the 11th predicted frame: 5.078125%
Tokens Correct (%) in the 12th predicted frame: 4.6875%
Tokens Correct (%) in the 13th predicted frame: 4.296875%
Tokens Correct (%) in the 14th predicted frame: 2.734375%
Tokens Correct (%) in the 15th predicted frame: 1.953125%
Tokens Correct (%) in the 16th predicted frame: 2.734375%
Tokens Correct (%) in the 17t

In [ ]:
# from google.colab import runtime
# runtime.unassign()

## Encoder-Decoder Transformer

In [ ]:
class OneshotFrameTransformer(nn.Module):
    num_embeddings: int
    num_heads: int
    num_layers: int
    embed_dim: int
    tokens_per_frame: int

    @nn.compact
    def __call__(self, history_tokens):
        # --- ENCODER ---
        # Takes the 4 history frames and creates a memory representation.
        history_pos_len = history_tokens.shape[-1]

        history_emb = nn.Embed(num_embeddings=self.num_embeddings, features=self.embed_dim)(history_tokens)
        history_pos_emb = self.param('history_pos_embedding', nn.initializers.normal(stddev=0.02), (1, history_pos_len, self.embed_dim))
        encoder_input = history_emb + history_pos_emb

        target_pe = self.param('target_pos_embedding', nn.initializers.normal(stddev=0.02), (1, self.tokens_per_frame, self.embed_dim))
        target_pe = target_pe.repeat(history_tokens.shape[0], axis=0)
        target_context = nn.Dense(self.embed_dim)(nn.gelu(nn.Dense(self.embed_dim * 4)(target_pe)))
        act = nn.Dense(self.embed_dim)(nn.gelu(nn.Dense(self.embed_dim * 4)(target_pe)))

        context = jnp.concatenate([encoder_input, target_context], axis=1)

        for _ in range(self.num_layers):
            # Self-attention on target tokens + Cross-attention to the encoder's memory
            act += nn.MultiHeadDotProductAttention(
                num_heads=self.num_heads, dtype=jnp.bfloat16
            )(inputs_q=act, inputs_kv=context)
            # Feed-forward layer
            act += nn.Dense(self.embed_dim)(nn.gelu(nn.Dense(self.embed_dim * 4)(act)))

        logits = nn.Dense(features=self.num_embeddings)(act)
        return logits


@partial(jax.jit, static_argnums=(5, 6))
def train_step(transformer_state, vqvae_state, points, velocities, key, model, history_len):
    # @partial(jax.jit, static_argnums=(3,))
    # def train_step(state, batch, key, model):
    tokens = get_batch(points, velocities, history_len, vqvae_state)
    history_tokens = tokens[:, :history_len * tokens_per_frame]
    target_tokens = tokens[:, history_len * tokens_per_frame:]

    dropout_key = jax.random.fold_in(key, transformer_state.step)

    def loss_fn(params):
        # The model only takes the history as input now
        logits = model.apply(
            {'params': params},
            history_tokens,
            rngs={'dropout': dropout_key}
        )
        # The loss is calculated between the output logits and the actual target frame
        loss = optax.softmax_cross_entropy_with_integer_labels(
            logits=logits, labels=target_tokens
        ).mean()
        return loss

    loss, grads = jax.value_and_grad(loss_fn)(transformer_state.params)
    transformer_state = transformer_state.apply_gradients(grads=grads)
    return transformer_state, loss


@partial(jax.jit, static_argnums=(1, 2, 5))
def generate_trajectory(transformer_state, history_len, tokens_per_frame, key, initial_history_tokens, num_predictions):
    """
    Generates a trajectory by predicting one full frame at a time.
    """
    def generate_next_frame_and_update_history(history_buffer, key):
        # Predict the logits for the next frame in a single forward pass
        logits = transformer_state.apply_fn(
            {'params': transformer_state.params},
            history_buffer[jnp.newaxis, :], # Add batch dim
            rngs={'dropout': key}
        )

        # Sample from the logits to get the new frame's tokens
        predicted_frame = jax.random.categorical(key, logits, axis=-1).squeeze(0)

        # Create the new history for the next iteration by sliding the window
        new_history_buffer = jnp.concatenate([history_buffer[tokens_per_frame:], predicted_frame])

        return new_history_buffer, predicted_frame

    # Run the scan to generate all predicted frames
    keys = jax.random.split(key, num_predictions)
    _, all_predicted_frames = jax.lax.scan(
        generate_next_frame_and_update_history,
        initial_history_tokens,
        keys
    )

    return all_predicted_frames

In [ ]:
# --- Initialize Transformer Model ---
print("🤖 Initializing Transformer model...")
oneshot_model = OneshotFrameTransformer(
    num_embeddings=vq_num_embeddings, num_heads=transformer_num_heads, num_layers=transformer_num_layers,
    embed_dim=transformer_embed_dim, tokens_per_frame=tokens_per_frame
)
key, transformer_key, dropout_key = jax.random.split(key, 3)
dummy_transformer_input = jnp.ones((1, history_len * tokens_per_frame), dtype=jnp.int32)
oneshot_params = oneshot_model.init(
    {'params': transformer_key, 'dropout': dropout_key}, dummy_transformer_input
)['params']
oneshot_state = train_state.TrainState.create(
    apply_fn=oneshot_model.apply, params=oneshot_params, tx=optax.adam(learning_rate=transformer_lr)
)

# --- Define an absolute directory for checkpoints ---
os_ckpt_dir = os.path.abspath('./oneshot_transformer_checkpoints')
os.makedirs(os_ckpt_dir, exist_ok=True)

# --- Restore checkpoint if one exists ---
# This will load the latest checkpoint and update the state object in place.
# If no checkpoint exists, it returns the original state object.
oneshot_state = checkpoints.restore_checkpoint(ckpt_dir=os_ckpt_dir, target=oneshot_state)
print(f"Resuming training from step: {int(oneshot_state.step)}")

🤖 Initializing Transformer model...
Resuming training from step: 0


In [ ]:
# --- Phase: Train the Transformer ---
print("🚀 Starting Transformer training...")
for epoch in range(transformer_epochs):
    data_loader = create_point_cloud_dataset(
        transformer_batch_size, history_len, num_points
    )
    pbar = tqdm(data_loader, desc=f"Epoch {epoch + 1}/{transformer_epochs}")
    total_loss = 0
    for i, (points, velocities) in enumerate(pbar):
        key, train_key = jax.random.split(key)
        oneshot_state, loss = train_step(
            oneshot_state, vqvae_state, points, velocities, train_key, oneshot_model,
            history_len
        )
        total_loss += loss
        pbar.set_postfix(loss=f"{loss:.4f}")

    if epoch % 50 == 0:
        # --- Save checkpoint at the end of the epoch ---
        checkpoints.save_checkpoint(
            ckpt_dir=os_ckpt_dir,
            target=oneshot_state,
            step=oneshot_state.step,
            overwrite=False, # Set to True to save only the latest checkpoint
            keep=3          # Keep the 3 most recent checkpoints
        )
    print(f"Epoch {epoch + 1} Avg Loss: {total_loss / (i+1):.4f}")

🚀 Starting Transformer training...


Epoch 1/1000: 39it [00:16,  2.32it/s, loss=4.4526]


Epoch 1 Avg Loss: 4.4385


Epoch 2/1000: 39it [00:08,  4.55it/s, loss=4.4381]


Epoch 2 Avg Loss: 4.4399


...

Epoch 998 Avg Loss: 1.5465


Epoch 999/1000: 39it [00:08,  4.47it/s, loss=1.5553]


Epoch 999 Avg Loss: 1.5498


Epoch 1000/1000: 39it [00:08,  4.48it/s, loss=1.5361]

Epoch 1000 Avg Loss: 1.5475


In [91]:
oneshot_predicted_token_grids = generate_trajectory(
    oneshot_state, history_len, tokens_per_frame, key, all_tokens[:history_len].flatten(), len(all_tokens)-history_len
)

predicted_frames, check_decoder = animate_prediction(
    'onehot_predicted_traj.mp4',
    orig_imgs,
    all_frames,
    all_tokens,
    oneshot_predicted_token_grids,
    get_tokens_fn,
    decode_fn,
    history_len,
)

🎥 Generating prediction and creating animation...
✅ Animation saved to 'onehot_predicted_traj.mp4'


In [92]:
oneshot_metrics = metrics(
    oneshot_predicted_token_grids,
    all_tokens[history_len:],
    predicted_frames,
    check_decoder[history_len:],
)
np.savez_compressed("frame_autoregressive_metrics.npz", **oneshot_metrics)

Tokens Correct (%) in the 0th predicted frame: 41.40625%
Tokens Correct (%) in the 1th predicted frame: 20.3125%
Tokens Correct (%) in the 2th predicted frame: 16.40625%
Tokens Correct (%) in the 3th predicted frame: 13.671875%
Tokens Correct (%) in the 4th predicted frame: 12.890625%
Tokens Correct (%) in the 5th predicted frame: 11.71875%
Tokens Correct (%) in the 6th predicted frame: 8.984375%
Tokens Correct (%) in the 7th predicted frame: 7.421875%
Tokens Correct (%) in the 8th predicted frame: 7.03125%
Tokens Correct (%) in the 9th predicted frame: 6.640625%
Tokens Correct (%) in the 10th predicted frame: 6.25%
Tokens Correct (%) in the 11th predicted frame: 6.25%
Tokens Correct (%) in the 12th predicted frame: 5.859375%
Tokens Correct (%) in the 13th predicted frame: 4.6875%
Tokens Correct (%) in the 14th predicted frame: 5.078125%
Tokens Correct (%) in the 15th predicted frame: 4.6875%
Tokens Correct (%) in the 16th predicted frame: 4.6875%
Tokens Correct (%) in the 17th predict

### Plot

In [93]:
transformer_metrics = {}
for fn in os.listdir():
  if fn.endswith("_metrics.npz"):
    transformer_metrics[fn.split("_metrics.npz")[0]] = dict(np.load(fn))

# 1. Get the structure of the dictionary
outer_keys = list(transformer_metrics.keys())
# Get inner keys from the first entry, assuming the structure is consistent
inner_keys = list(transformer_metrics[outer_keys[0]].keys())
num_subplots = len(inner_keys)

# 2. Create a figure with one subplot for each inner key
fig, axes = plt.subplots(1, num_subplots, figsize=(8 * num_subplots, 5))
# If there's only one subplot, `plt.subplots` returns a single axis object
if num_subplots == 1:
    axes = [axes]

# 3. Iterate through each metric (e.g., 'Tokens Correct') to create a subplot
for ax, inner_key in zip(axes, inner_keys):

    # 4. On each subplot, iterate through each model type (e.g., 'autoregressive') to plot its line
    for outer_key in outer_keys:
        # Get the data array for the current line
        data = transformer_metrics[outer_key][inner_key]
        x_axis = np.arange(len(data))

        # Plot the data, using the outer key as the label for the legend
        ax.plot(x_axis, data, label=outer_key.capitalize(), marker='o', linestyle='-')

    # 5. Customize the subplot
    ax.set_title(f'Metric: "{inner_key}"', fontsize=14)
    ax.set_xlabel('Frame #', fontsize=12)
    ax.set_ylabel('Value', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend(title="Model Type")

# 6. Final adjustments and display
plt.suptitle('Comparison of Transformer Metrics', fontsize=16, y=1.02)
plt.tight_layout()
plt.savefig("metrics.png")
plt.close(fig)

In [ ]:
# from google.colab import runtime
# runtime.unassign()

In [79]:
checkpoints.save_checkpoint(
            ckpt_dir=os_ckpt_dir,
            target=oneshot_state,
            step=oneshot_state.step,
            overwrite=False, # Set to True to save only the latest checkpoint
            keep=3          # Keep the 3 most recent checkpoints
        )

'/content/drive/MyDrive/Colab Notebooks/VQ-Dynamics/oneshot_transformer_checkpoints/checkpoint_40399'

In [3]:
# !zip -r all_files.zip .

  adding: encode.py (deflated 69%)
  adding: __pycache__/ (stored 0%)
  adding: __pycache__/vqvae.cpython-311.pyc (deflated 53%)
  adding: images_jax/ (stored 0%)
  adding: images_jax/orig_frame_0000.png (deflated 4%)
  adding: images_jax/recon_frame_0000.png (deflated 3%)
  adding: images_jax/recon_frame_0002.png (deflated 4%)
  adding: images_jax/orig_frame_0001.png (deflated 3%)
  adding: images_jax/orig_frame_0003.png (deflated 2%)
  adding: images_jax/recon_frame_0001.png (deflated 4%)
  adding: images_jax/orig_frame_0002.png (deflated 3%)
  adding: images_jax/recon_frame_0003.png (deflated 3%)
  adding: images_jax/orig_frame_0004.png (deflated 2%)
  adding: images_jax/recon_frame_0004.png (deflated 3%)
  adding: images_jax/recon_frame_0005.png (deflated 4%)
  adding: images_jax/orig_frame_0005.png (deflated 3%)
  adding: images_jax/orig_frame_0006.png (deflated 2%)
  adding: images_jax/recon_frame_0006.png (deflated 4%)
  adding: images_jax/orig_frame_0007.png (deflated 2%)
  add

In [ ]:
# from google.colab import files
# files.download('all_files.zip')